# ST7735 Display Project: Extensible Serial Monitor Calibration and Display Management System Design Proposal

This notebook documents the design proposal for an extensible, serial monitor-driven calibration and display management system for the ST7735 project. It covers data structures, menu design, program flow, extensibility, and implementation steps.

## 1. Data Structures

**Display orientation enum:**
```cpp
enum DisplayOrientation {
  PORTRAIT,           // 0° - pins on top (128w x 160h)
  LANDSCAPE,          // 90° CW - pins on left (160w x 128h)
  REVERSE_PORTRAIT,   // 180° - pins on bottom (128w x 160h)
  REVERSE_LANDSCAPE   // 270° CW - pins on right (160w x 128h)
};
```

**Display physical constants:**
```cpp
// ST7735 1.8" display maximum dimensions
const int DISPLAY_MAX_WIDTH = 160;
const int DISPLAY_MAX_HEIGHT = 128;

// Initial frame (full panel) - will be adjusted during calibration
const int INITIAL_FRAME_TOP = 0;
const int INITIAL_FRAME_BOTTOM = DISPLAY_MAX_HEIGHT - 1;  // 127
const int INITIAL_FRAME_LEFT = 0;
const int INITIAL_FRAME_RIGHT = DISPLAY_MAX_WIDTH - 1;    // 159
const int INITIAL_FRAME_THICKNESS = 2;
```

**DisplayState struct/class (one per display):**
```cpp
enum DisplayContentType {
  WAITING_IMAGE,
  BITMAP_IMAGE,
  CALIBRATION_FRAME,
  VOLATILE_DATA,
  OTHER
};

struct DisplayState {
  // Frame parameters
  int frameTop;
  int frameBottom;
  int frameLeft;
  int frameRight;
  int frameThickness;
  DisplayOrientation frameOrientation;
  
  // Calculated properties (derived from frame parameters)
  int frameWidth;   // frameRight - frameLeft + 1
  int frameHeight;  // frameBottom - frameTop + 1
  int frameCenterX; // frameLeft + frameWidth / 2
  int frameCenterY; // frameTop + frameHeight / 2

  // Display content type
  DisplayContentType currentContent;

  // Calibration aids
  bool showDiagonalLine;  // Draw diagonal from [0,0] to calculated center

  // Last bitmap or data (optional, for restoration)
  // e.g., uint16_t lastBitmap[WIDTH][HEIGHT];

  // Calibration parameters (if needed)
  // ...other extensible fields...
};
```

**Initialization function:**
```cpp
void initializeDisplay(DisplayState &display, DisplayOrientation orientation) {
  display.frameTop = INITIAL_FRAME_TOP;
  display.frameBottom = INITIAL_FRAME_BOTTOM;
  display.frameLeft = INITIAL_FRAME_LEFT;
  display.frameRight = INITIAL_FRAME_RIGHT;
  display.frameThickness = INITIAL_FRAME_THICKNESS;
  display.frameOrientation = orientation;
  display.currentContent = WAITING_IMAGE;
  display.showDiagonalLine = false;
  
  // Calculate derived values
  display.frameWidth = display.frameRight - display.frameLeft + 1;
  display.frameHeight = display.frameBottom - display.frameTop + 1;
  display.frameCenterX = display.frameLeft + display.frameWidth / 2;
  display.frameCenterY = display.frameTop + display.frameHeight / 2;
}
```

**Global array for all displays:**
```cpp
const int NUM_DISPLAYS = 2; // Hard-coded for now, extensible later
DisplayState displays[NUM_DISPLAYS];

void setup() {
  // Initialize displays with their physical orientation
  initializeDisplay(displays[0], LANDSCAPE);  // Example: Display 1 is landscape
  initializeDisplay(displays[1], PORTRAIT);   // Example: Display 2 is portrait
}
```

**Drawing the calibration frame with diagonal:**
```cpp
void drawCalibrationFrame(DisplayState &display, Adafruit_ST7735 &tft) {
  uint16_t frameColor = ST77XX_WHITE;
  uint16_t diagonalColor = ST77XX_YELLOW;
  
  // Draw frame rectangle
  for (int i = 0; i < display.frameThickness; i++) {
    tft.drawRect(
      display.frameLeft + i,
      display.frameTop + i,
      display.frameWidth - (2 * i),
      display.frameHeight - (2 * i),
      frameColor
    );
  }
  
  // Draw diagonal line from [0,0] to calculated center (if enabled)
  if (display.showDiagonalLine) {
    tft.drawLine(0, 0, display.frameCenterX, display.frameCenterY, diagonalColor);
  }
}
```

## 2. Serial Monitor Menu System

**Main Menu:**
```
Display Calibration & Management
-------------------------------
Select display to calibrate:
  1. Display 1 (Landscape)
  2. Display 2 (Portrait)
  ...
  S. Save and exit
Enter choice:
```

**Display Calibration Menu (for selected display):**
```
Display X Calibration (Orientation: LANDSCAPE)
---------------------------------------------
1. Move individual frame line
2. Move entire frame
3. Adjust frame thickness
4. Toggle diagonal line (currently: OFF)
5. Change orientation
6. Show current parameters
7. Restore previous display content
8. Return to main menu
Enter choice:
```

**Move Individual Frame Line:**
```
Select line to move:
  1. Top
  2. Bottom
  3. Left
  4. Right
Enter choice:
Amount to move (+/- pixels):
```

**Move Entire Frame:**
```
Direction:
  1. Up
  2. Down
  3. Left
  4. Right
Amount to move (+/- pixels):
```

**Adjust Frame Thickness:**
```
Enter new thickness (pixels):
```

**Toggle Diagonal Line:**
```
Diagonal line from [0,0] to center [toggled ON/OFF]
```

**Change Orientation:**
```
Select orientation:
  1. Portrait (pins on top) - 128x160
  2. Landscape (pins on left) - 160x128
  3. Reverse Portrait (pins on bottom) - 128x160
  4. Reverse Landscape (pins on right) - 160x128
Enter choice:
```

**Show Current Parameters:**
```
Display X Parameters:
  Orientation: LANDSCAPE (160x128)
  Top: <value>
  Bottom: <value>
  Left: <value>
  Right: <value>
  Width: <value>
  Height: <value>
  Center: (<centerX>, <centerY>)
  Thickness: <value>
  Diagonal Line: ON/OFF
  Content: <WAITING_IMAGE/BITMAP_IMAGE/etc.>
```

**Restore Previous Display Content:**
- Restores last bitmap or data if not currently showing waiting image.

## 2.1 Menu Command Handlers

**Toggle Diagonal Line Handler:**
```cpp
void toggleDiagonalLine(DisplayState &display, Adafruit_ST7735 &tft) {
  display.showDiagonalLine = !display.showDiagonalLine;
  
  // Redraw the calibration frame
  tft.fillScreen(ST77XX_BLACK);
  drawCalibrationFrame(display, tft);
  
  SerialUSB.print("Diagonal line ");
  SerialUSB.println(display.showDiagonalLine ? "ON" : "OFF");
  SerialUSB.print("Line drawn from [0,0] to center [");
  SerialUSB.print(display.frameCenterX);
  SerialUSB.print(",");
  SerialUSB.print(display.frameCenterY);
  SerialUSB.println("]");
}
```

**Change Orientation Handler:**
```cpp
void changeOrientation(DisplayState &display, Adafruit_ST7735 &tft, DisplayOrientation newOrientation) {
  display.frameOrientation = newOrientation;
  
  // Reset to default frame for the new orientation
  display.frameTop = INITIAL_FRAME_TOP;
  display.frameBottom = INITIAL_FRAME_BOTTOM;
  display.frameLeft = INITIAL_FRAME_LEFT;
  display.frameRight = INITIAL_FRAME_RIGHT;
  
  // Recalculate derived values
  display.frameWidth = display.frameRight - display.frameLeft + 1;
  display.frameHeight = display.frameBottom - display.frameTop + 1;
  display.frameCenterX = display.frameLeft + display.frameWidth / 2;
  display.frameCenterY = display.frameTop + display.frameHeight / 2;
  
  // Apply rotation to TFT
  switch (newOrientation) {
    case PORTRAIT:
      tft.setRotation(0);
      break;
    case LANDSCAPE:
      tft.setRotation(1);
      break;
    case REVERSE_PORTRAIT:
      tft.setRotation(2);
      break;
    case REVERSE_LANDSCAPE:
      tft.setRotation(3);
      break;
  }
  
  // Redraw
  tft.fillScreen(ST77XX_BLACK);
  drawCalibrationFrame(display, tft);
  
  SerialUSB.print("Orientation changed to: ");
  printOrientation(display.frameOrientation);
}

void printOrientation(DisplayOrientation orientation) {
  switch (orientation) {
    case PORTRAIT:
      SerialUSB.println("PORTRAIT (128x160)");
      break;
    case LANDSCAPE:
      SerialUSB.println("LANDSCAPE (160x128)");
      break;
    case REVERSE_PORTRAIT:
      SerialUSB.println("REVERSE_PORTRAIT (128x160)");
      break;
    case REVERSE_LANDSCAPE:
      SerialUSB.println("REVERSE_LANDSCAPE (160x128)");
      break;
  }
}
```

**Update Calculated Values (call after any frame parameter change):**
```cpp
void updateCalculatedValues(DisplayState &display) {
  display.frameWidth = display.frameRight - display.frameLeft + 1;
  display.frameHeight = display.frameBottom - display.frameTop + 1;
  display.frameCenterX = display.frameLeft + display.frameWidth / 2;
  display.frameCenterY = display.frameTop + display.frameHeight / 2;
}
```

## 3. Program Flow

- On startup, initialize all `DisplayState` structs with default values.
- Always track what is currently displayed for each display.
- When entering calibration mode, save current display content if not waiting image.
- All menu operations update the relevant `DisplayState` and redraw the frame on the selected display.
- When calibration is complete, print all parameters to serial for user to save/copy.
- Main program uses these parameters for future data display.

## 4. Extensibility Considerations

- All display management is abstracted via `DisplayState` and enums, allowing easy addition of new display types or content.
- Menu system is modular: new calibration steps or display types can be added with minimal changes.
- Serial commands and menu options are versioned for future compatibility.
- Calibration parameters are stored in a way that supports multiple displays and future expansion.

## 5. Implementation Steps

1. Define `DisplayState` struct/class and `DisplayContentType` enum in code.
2. Implement serial monitor menu system for calibration and display management.
3. Add logic to update display state and redraw frames based on menu actions.
4. Store calibration parameters and print them to serial for user to save.
5. Integrate calibration logic with main program flow.
6. Test with multiple displays and content types to ensure extensibility.

## 6. Saving and Restoring the Current Display Content (before calibration)

When a user enters calibration mode we should preserve the currently shown content so it can be restored after calibration (or if the user cancels). This cell documents a safe, memory-aware pattern you can drop into the sketch and into the serial/menu workflow.

Goals
- Capture exactly what is shown on a single display (the pixel data and its display offset) in RAM so it can be restored later.
- Keep the API simple: saveSnapshot() and restoreSnapshot().
- Respect RAM limits on the Due and avoid fragmentation and leaks.
- Provide serial/menu commands so the user can trigger save/restore from the monitor.

Constraints & memory sizing
- The displays are 160x128 and use RGB565 (2 bytes per pixel). One full-screen buffer requires ~160*128*2 = 40,960 bytes (~40 KB).
- Arduino Due has ~96 KB SRAM; storing one full buffer is acceptable, two would be borderline depending on other memory usage.
- If you need frequent snapshotting or multiple snapshots, prefer external FRAM or store only the rectangular region of interest.

Data shape (C++ types - design only)

```cpp
// Minimal metadata for a snapshot
struct DisplaySnapshot {
  uint16_t width;      // width of captured rectangle
  uint16_t height;     // height of captured rectangle
  int16_t  offsetX;    // where it was drawn on the display
  int16_t  offsetY;
  // Followed by width*height uint16_t pixels in RGB565 order
};
```

Recommended runtime pattern
1. Before entering calibration (if current content != waiting-image): call saveSnapshotForDisplay(displayIndex).
2. The function captures the displayed rectangle (or full usable area), allocating a single contiguous block (metadata + pixels).
3. When restoring, validate the stored metadata and draw pixels back to the display at the saved offset.
4. Free the block when no longer needed (or keep it until an explicit discard).

Serial commands (simple)
- `SAVE_DISPLAY` — save the currently visible content to RAM
- `RESTORE_DISPLAY` — restore the saved snapshot to the display
- `DISCARD_DISPLAY` — free the saved snapshot

Example implementation sketch (drop into your design document; adapt to project APIs)

```cpp
// Allocate a snapshot for a rectangular region (w x h) and capture pixels
uint8_t* captureSnapshot(int x, int y, int w, int h) {
  size_t pixels = (size_t)w * (size_t)h;
  size_t meta = sizeof(DisplaySnapshot);
  size_t bytes = meta + pixels * sizeof(uint16_t);
  // Simple sanity check
  if (bytes > 60 * 1024) return nullptr; // avoid oversized allocations
  uint8_t *buf = (uint8_t*)malloc(bytes);
  if (!buf) return nullptr;

  DisplaySnapshot *hdr = (DisplaySnapshot*)buf;
  hdr->width = w;
  hdr->height = h;
  hdr->offsetX = x;
  hdr->offsetY = y;

  // Read pixels into buffer row-by-row. Many ST7735 libraries don't provide a fast 'readPixel' -
  // instead keep the pixel source (your generator) or capture when receiving pixels from serial.
  // If you must read from the display, be aware performance may be slow or unsupported.
  uint16_t *pixelsPtr = (uint16_t*)(buf + meta);
  for (int r = 0; r < h; ++r) {
    for (int c = 0; c < w; ++c) {
      // If you maintain the incoming bitmap buffer already (recommended), copy from that.
      // Otherwise try tft.readPixel(x + c, y + r) if supported by your driver.
      pixelsPtr[r*w + c] = tft.readPixel(x + c, y + r); // pseudo-call, check library support
    }
  }
  return buf;
}

// Restore snapshot
bool restoreSnapshot(uint8_t *buf) {
  if (!buf) return false;
  DisplaySnapshot *hdr = (DisplaySnapshot*)buf;
  uint16_t *pixelsPtr = (uint16_t*)(buf + sizeof(DisplaySnapshot));
  for (int r = 0; r < hdr->height; ++r) {
    for (int c = 0; c < hdr->width; ++c) {
      int dx = hdr->offsetX + c;
      int dy = hdr->offsetY + r;
      if (isWithinBounds(dx, dy)) {
        tft.drawPixel(dx, dy, pixelsPtr[r*hdr->width + c]);
      }
    }
  }
  return true;
}

// Free snapshot
void freeSnapshot(uint8_t *buf) {
  if (buf) free(buf);
}
```

Practical notes
- The Adafruit ST7735 library historically does not implement a `readPixel()` method for all configurations. The reliable approach is to capture pixels as they are received (during bitmap upload) and keep that buffer in RAM (this is the cleanest and fastest). If you capture on receipt, you avoid the slow/unsupported read-back path.
- Keep snapshots small: capture only the rectangle you need (e.g., the usable area or the last bitmap) rather than the whole panel when possible.
- Use a single allocation for metadata+pixels to keep fragmentation low.
- Always check for `malloc()` returning null and gracefully fall back to 'no snapshot' behavior.

Integration with the menu flow
- When the user selects "Calibrate display X":
  - If the display `currentContent` != `WAITING_IMAGE`, send a `SAVE_DISPLAY` command (or call the API) to capture the content.
  - Run calibration actions (draw frames, move lines, etc.).
  - If the user cancels or finishes and wants to restore prior content, call `RESTORE_DISPLAY`.
  - On successful finalization, optionally discard the saved snapshot or keep it for future restores.

Next steps I can take for you
- Insert a concrete, tested `capture`/`restore` implementation into `src/main.cpp` that captures incoming bitmap data in RAM (recommended approach). I will ensure it uses one contiguous allocation and add `SAVE_DISPLAY`/`RESTORE_DISPLAY` serial commands wired into the menu.
- Or, if you prefer, I can add the above design notes as a dedicated code file (e.g., `src/display_snapshot.h` + `src/display_snapshot.cpp`) and integrate it with the menu.

Tell me which of the two you'd like: implement the tested capture/restore code in the sketch now, or keep this as design notes only in the notebook?

## 7. Serial Port Configuration and Auto-Detection

The calibration tools and any Python programs communicating with the Arduino Due should use the **Native USB port** to avoid resets during data transfer.

### Port Types

The Arduino Due has two USB ports:

| Port Type | Device Example | Arduino Code | Behavior |
|-----------|----------------|--------------|----------|
| **Native USB** | `/dev/ttyACM1` | `SerialUSB` | Does NOT reset on connection ✓ |
| **Programming Port** | `/dev/ttyACM0` | `Serial` | Resets when DTR toggles |

### Using the st7735_tools Module

The project includes a `st7735_tools` package that automatically detects Arduino Due ports:

#### Python Import
```python
from st7735_tools import (
    get_native_usb_port,
    get_programming_port,
    get_preferred_port,
    print_arduino_due_info
)
```

#### Auto-Detection Example
```python
# Get Native USB port (recommended)
port = get_native_usb_port()
if port:
    print(f"Native USB: {port}")
    # Use this port for SerialUSB communication
    ser = serial.Serial(port, 115200)
else:
    print("Native USB port not found")

# Or use smart fallback
port = get_preferred_port(prefer_native=True)
```

#### Detection Info
```python
# Show all detected ports
print_arduino_due_info()
```

### Arduino Code Requirements

When using the Native USB port, the Arduino code must use `SerialUSB` instead of `Serial`:

```cpp
void setup() {
  SerialUSB.begin(115200);
  // Do NOT wait for SerialUSB connection if you want standalone operation
  // while (!SerialUSB) { } // Only use this for debugging
  
  SerialUSB.println("Arduino Due ready on Native USB");
}

void loop() {
  if (SerialUSB.available()) {
    String command = SerialUSB.readStringUntil('\n');
    // Process calibration commands
  }
}
```

### Integration with Calibration Tools

Any Python calibration tool should:

1. Import `st7735_tools` for port detection
2. Auto-detect the Native USB port by default
3. Allow manual port override if needed
4. Display warnings if using the Programming port

Example:
```python
from st7735_tools import get_native_usb_port
import serial

# Auto-detect
port = get_native_usb_port()
if not port:
    print("Error: Native USB port not found")
    print("Please connect Arduino Due Native USB port")
    exit(1)

print(f"Connecting to {port}...")
ser = serial.Serial(port, 115200, timeout=2)
time.sleep(2)  # Wait for Arduino to initialize

# Send calibration commands
ser.write(b"MENU\n")
```

### Module Location

The `st7735_tools` package is located at:
```
ST7735-Display-Project/
  st7735_tools/
    __init__.py
    serial_utils.py
    README.md
```

Import it in your scripts from the project root directory.